### Linear regression for data with multiple features
Linear regression with sklearn on the auto-mpg dataset

***
#### Environment
`conda activate sklearn-env`

***
#### Goals
- Train linear regression with sklearn
- Predict values from test dataset and compare with test labels

***
#### References
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

#### Basic python imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random 


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

#### Dataset load from CSV located on UCI website.

http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data  
If the URL does not work the dataset can be loaded from the data folder `./data/auto-mpg.data`.

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)
dataset = raw_dataset.copy()
dataset.tail(2)

#### Data preparation

Split data in `training` and `test` datasets


In [ ]:
dataset = dataset.dropna().copy()[['MPG', 'Weight']]

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

#### Traing sklean linear regression algorithm (based on training datasets)

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regressor = LinearRegression().fit(train_features, train_labels)

#### Predict values from test dataset and compare with test labels

In [ ]:
scored_test = linear_regressor.predict(test_features)
test_dataset['Predicted']=scored_test

In [ ]:
from sklearn.metrics import mean_squared_error

new_df = test_dataset[['Predicted', 'MPG']].copy()
rmse = mean_squared_error(new_df[['MPG']], new_df[['Predicted']])
print("RMSE is %f for %d records" % (rmse, len(test_features)))

new_df.reset_index(inplace=True, drop=True)
new_df.head(10)

In [ ]:
test_digit = random.randint(1, len(scored_test))
test_dataset.iloc[[test_digit]]

#### Plot the linear regression

In [ ]:
# https://machinelearningmastery.com/prediction-intervals-for-machine-learning/
from numpy import sum as arraysum
from numpy import sqrt
from numpy import mean

sum_errs = arraysum((test_dataset['MPG'] - test_dataset['Predicted'])**2)
stdev = sqrt(1/(len(test_dataset['MPG'])-2) * sum_errs)
interval = 1.96 * stdev

stdev

In [ ]:
x = test_dataset['Weight'].iloc[0]
y = test_dataset['Predicted'].iloc[0]
y_base = test_dataset['MPG'].iloc[0]

print('Predicted value is %.3f' % y)
print('Actual value is %.3f' % y_base)
print('95%% likelihood that the true value is between %.3f and %.3f' % (y-interval, y+interval))

In [ ]:
plt.figure(figsize=(10, 6))

plt.scatter(test_dataset['Weight'], test_dataset['MPG'])
plt.plot(test_dataset['Weight'], test_dataset['Predicted'], color='red')
plt.errorbar( x, y, yerr=interval, color='black', fmt='o')

plt.legend(['Prediction', 'Actual Value'])
plt.xlabel('Weight')
plt.ylabel('MPG')

plt.title("Linear Regression with Prediction Interval")